In [1]:
import os
import torch
import torchvision
from torchvision import transforms
import cv2 as cv
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd

In [2]:
import torch.onnx as torch_onnx
import onnx
import cv2
import onnxruntime as ort

In [4]:
# Moving to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
os.path.abspath('')

'/Users/arth'

In [3]:
!ls

ASL-Translator                  Postman
Adlm                            Public
Applications                    PycharmProjects
Creative Cloud Files            Shit
Desktop                         Spring 2019
Documents                       Untitled.ipynb
Downloads                       Untitled1.ipynb
Essay 1 Outline.docx            Virtual Machines.localized
Library                         code
Movies                          cs61b-software
Music                           dev
New                             env
Normal Processing.ipynb         opt
Parallel Processing_Final.ipynb repo
Pictures                        w210-fall19-deepfake.pem


In [30]:
data_frame = pd.read_csv("sign_mnist_train.csv")
length = data_frame.shape[0]

In [31]:
data_frame.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [32]:
# Data augmentations

transform = transforms.Compose([transforms.RandomResizedCrop(28, scale=(0.8, 1.2)), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
transforms.ToTensor()])

In [35]:
# Implementing the custom dataset and dataloader

class ImageSet(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.data_frame = dataframe
        self.transform = transform

    
    def __getitem__(self, idx):
        """Fetching items at given idx through Dataloader"""
        
        label = int(self.data_frame.iloc[idx, 0])
        item = self.data_frame.iloc[idx, 1:].to_numpy()
        item = item.reshape((28, 28, 1))
        if self.transform:
          item = self.transform(item)
        sample = {'item': item, "label": label}
        return sample
    
    def __len__(self):
        return len(self.data_frame)
    


%time dataSet = ImageSet(data_frame)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


In [36]:
%time dataLoader = DataLoader(dataSet, batch_size=4)

CPU times: user 348 µs, sys: 2.71 ms, total: 3.06 ms
Wall time: 6.82 ms


In [22]:
# Implementing the CNN

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1 , 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 6, 3)
        self.conv3 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 48)
        self.fc3 = nn.Linear(48, 25)
    
    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = F.leaky_relu(self.conv1(x))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = self.pool(F.leaky_relu(self.conv3(x)))
        x = x.reshape(-1, 16 * 5 * 5)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x      
        

In [38]:
net = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [39]:
torch.autograd.set_detect_anomaly(True)
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['item'], data['label']
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        # print statistics
        running_loss += loss.item()
    # Calculates average loss over epoch
    print("Epoch " + str(epoch), " Loss: ", round(running_loss/length, 2))

torch.save(net.state_dict(), "checkpoint.pth")

print('Finished Training')

Epoch 0  Loss:  0.2
Epoch 1  Loss:  0.0
Epoch 2  Loss:  0.0
Epoch 3  Loss:  0.0
Epoch 4  Loss:  0.0
Epoch 5  Loss:  0.0
Epoch 6  Loss:  0.0
Epoch 7  Loss:  0.0
Epoch 8  Loss:  0.0
Epoch 9  Loss:  0.0
Finished Training


In [6]:
!ls

ASL-Translator                  Postman
Adlm                            Public
Applications                    PycharmProjects
Creative Cloud Files            Shit
Desktop                         Spring 2019
Documents                       Untitled.ipynb
Downloads                       Untitled1.ipynb
Essay 1 Outline.docx            Virtual Machines.localized
Library                         code
Movies                          cs61b-software
Music                           dev
New                             env
Normal Processing.ipynb         opt
Parallel Processing_Final.ipynb repo
Pictures                        w210-fall19-deepfake.pem


In [13]:
!ls

ASL-Translator                  Postman
Adlm                            Public
Applications                    PycharmProjects
Creative Cloud Files            Shit
Desktop                         Spring 2019
Documents                       Untitled.ipynb
Downloads                       Untitled1.ipynb
Essay 1 Outline.docx            Virtual Machines.localized
Library                         code
Movies                          cs61b-software
Music                           dev
New                             env
Normal Processing.ipynb         opt
Parallel Processing_Final.ipynb repo
Pictures                        w210-fall19-deepfake.pem


In [20]:
os.chdir("asl-dataset")

In [17]:
os.getcwd()

'/Users/arth/Desktop'

In [23]:
# Load pretrained model weights
model_url = torch.load("checkpoint.pth")
batch_size = 1    # just a random number

# Initialize model with the pretrained weights
net = ConvNet().float().eval()
net.load_state_dict(model_url)

# set the model to inference mode
net.eval()

ConvNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=48, bias=True)
  (fc3): Linear(in_features=48, out_features=25, bias=True)
)

In [24]:
fname = "signlanguage.onnx"
dummy = torch.randn(batch_size, 28, 28, batch_size)
torch.onnx.export(net, dummy, fname, input_names=['input'])

In [25]:
onnx_model = onnx.load("signlanguage.onnx")
onnx.checker.check_model(onnx_model)

In [26]:

def center_crop(frame):
    h, w, _ = frame.shape
    start = abs(h - w) // 2
    if h > w:
        return frame[start: start + w]
    return frame[:, start: start + h]

# constants
index_to_letter = list('ABCDEFGHIKLMNOPQRSTUVWXY')
mean = 0.485 * 255.
std = 0.229 * 255.

# create runnable session with exported model
ort_session = ort.InferenceSession("signlanguage.onnx")

cap = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # preprocess data
    frame = center_crop(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    x = cv2.resize(frame, (28, 28))
    x = (x - mean) / std

    x = x.reshape(1, 1, 28, 28).astype(np.float32)
    x = x.transpose(0, 3, 2, 1)
    y = ort_session.run(None, {'input': x})[0]

    index = np.argmax(y, axis=1)
    letter = index_to_letter[int(index)]

    cv2.putText(frame, letter, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 255, 0), thickness=2)
    cv2.imshow("Sign Language Translator", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range